## TEST Code for running and evaluating Steadiness / Cohesiveness and other metrics

#### Import packages & set dataset path

In [2]:
from helper import *
import sys
import importlib
import numpy as np

## Import Metrics
sys.path.append("/home/hj/vis2021_snc/metrics4mdp/metrics4mdp")
from provider import MDPMetricProvider

## Import SNC
sys.path.append("/home/hj/vis2021_snc/steadiness_cohesiveness")
import snc as sc




PATH_TO_DATASET = "../../dataset-generator/dataset/"

### Spheres TEST

In [ ]:
import time
min_dist = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
n_neighbors = [3]
klist=[40, 45, 50, 55, 60, 65, 70]


## reload SNC for the testing
importlib.reload(sc)
SNC = sc.SNC


def test_umap_dataset_sc(dataset,sample_rate, min_dist, n_neighbros, klist):
    for k in klist:
        steadiness_result = {}
        cohesiveness_result = {}
        for md in min_dist:
            s_temp_result = {}
            c_temp_result = {}
            for nn in n_neighbors:
                mnist_umap_path = make_path(PATH_TO_DATASET, dataset, "umap", sample_rate, [md, nn])
                emb_data, raw_data, label_data = get_data(mnist_umap_path)

                raw_data = np.array(raw_data)
                emb_data = np.array(emb_data)
                label_data = np.array(label_data)

                parameter = {
                    "k": k
                }

                start = time.time()
                snc = SNC(raw_data, emb_data, iteration=300, cluster_parameter = parameter)
                snc.fit()
                s_temp_result[nn] = snc.steadiness()
                c_temp_result[nn] = snc.cohesiveness()
                print(md,nn,"finished")
                print(s_temp_result[nn], c_temp_result[nn])
            steadiness_result[md] = s_temp_result
            cohesiveness_result[md] = c_temp_result
        data = {
            "steadiness": steadiness_result,
            "cohesivenss": cohesiveness_result
        }
        print("finished k =",k)
        import json
        with open("./" + dataset + "_" + str(sample_rate) + "_" + str(k) + "_result.json", "w") as outfile:
            json.dump(data, outfile)
            
# test_dataset_sc("spheres", 2, min_dist, n_neighbors, klist)

### Mammoth TEST

In [ ]:
import time
min_dist = [0.0]
n_neighbors = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
klist=[40, 45, 50, 55, 60, 65, 70]

print("START!!")
test_umap_dataset_sc("mammoth", 2, min_dist, n_neighbors, klist)

### PCA TEST

In [6]:
datasets = ["coil20"]
datasets = [ "mnist"]
sample_rate = 1
n_components = 60

importlib.reload(sc)
SNC = sc.SNC

nc_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
# k_list = [40, 45, 50, 55, 60]
k_list = [40]
# k_list = [50]

for dataset in datasets:

    mnist_pca_path = make_path(PATH_TO_DATASET, dataset, "pca", sample_rate, [n_components])
    emb_data, raw_data, label_data = get_data(mnist_pca_path)

    emb_data = np.array(emb_data)
    print("STARTED", dataset)
    for k in k_list:
        steadiness_result = {}
        cohesiveness_result = {}
        for nc in nc_list:
            current_emb_data = emb_data[:, nc:(nc+2)]

            parameter = {
                "k": k
            }
            snc = SNC(raw_data, current_emb_data, iteration=40, cluster_parameter = parameter)
            snc.fit()
            s_temp_result = snc.steadiness()
            c_temp_result = snc.cohesiveness()
            print(nc,"finished")
            print(s_temp_result, c_temp_result)
            steadiness_result[nc] = s_temp_result
            cohesiveness_result[nc] = c_temp_result

        data = {
            "steadiness": steadiness_result,
            "cohesivenss": cohesiveness_result
        }
        print("finished k =",k)
        import json
        with open("./" + dataset + "_" + str(sample_rate) + "_" + str(k) + "_result.json", "w") as outfile:
            json.dump(data, outfile)


STARTED mnist
0 finished
0.4750401253344163 0.697827572009867
1 finished
0.43764002193502427 0.6861229845072555
2 finished
0.4169505543649571 0.6905112151587339
3 finished
0.4082589018684457 0.6864570706687511
4 finished
0.41829446400457815 0.7239341550341282
5 finished
0.39403588352210306 0.7004580113424876
6 finished
0.39730029856061677 0.7119768326230083
7 finished
0.38480399823612466 0.7009690880079443
8 finished
0.39007082111687463 0.6965910320496547
9 finished
0.3886185720062567 0.7197024240338054
10 finished
0.38695474781935335 0.7051624533196235
11 finished
0.3881068033968339 0.6918635787466716
12 finished
0.3934749048701548 0.7050968125914623
13 finished
0.38392241456246645 0.7183443302092225
14 finished
0.38375573556284925 0.6955329242394506
15 finished
0.3808734963256134 0.6953361855487215
finished k = 40
